# KwikSort Re-Ranking with Estimating the ORACLE Axiom

The notebook below exemplifies how `ir_axioms` can be used to re-rank a result set in PyTerrier using the KwikSort algorithm and an estimation of the ORACLE axiom.
We use run files and qrels from the passage retrieval task of the TREC Deep Learning track in 2019 and 2020 as example (using BM25 as a baseline).
In this notebook, we first train the ORACLE axiom estimation using preferences inferred from 2019 qrels and topics.
Then, we re-rank using that trained `EstimatorAxiom` and evaluate nDCG@10, reciprocal rank, and average precision for the baseline and the re-ranked pipeline using PyTerrier's standard `Experiment` functionality.

## Preparation

Install the `ir_axioms` framework and [PyTerrier](https://github.com/terrier-org/pyterrier). In Google Colab, we do this automatically.

In [ ]:
from sys import modules

if 'google.colab' in modules:
    !pip install -q ir_axioms[experiments]>=1.0 python-terrier

## Datasets and Index
Using PyTerrier's `get_dataset()`, we load the MS MARCO passage ranking dataset.

In [ ]:
from pyterrier.datasets import get_dataset, Dataset

# Load dataset.
dataset_name = "msmarco-passage"
dataset: Dataset = get_dataset(f"irds:{dataset_name}")
dataset_train: Dataset = get_dataset(f"irds:{dataset_name}/trec-dl-2019/judged")
dataset_test: Dataset = get_dataset(f"irds:{dataset_name}/trec-dl-2020/judged")

Now define paths where we will store temporary files, datasets, and the search index.

In [ ]:
from pathlib import Path

cache_dir = Path("experiments/cache/")
index_dir = cache_dir / "indices" / dataset_name.split("/")[0]

If the index is not ready yet, now is a good time to create it and index the MS MARCO passages.
(Lean back and relax as this may take a while...)

In [ ]:
from pyterrier.index import IterDictIndexer

if not index_dir.exists():
    indexer = IterDictIndexer(str(index_dir.absolute()))
    indexer.index(
        dataset.get_corpus_iter(),
        fields=["text"]
    )

## Baseline Run

We use PyTerrier's `BatchRetrieve` to create a baseline search pipeline for retrieving with BM25 from the index we just created.

In [ ]:
from pyterrier.batchretrieve import BatchRetrieve

bm25 = BatchRetrieve(str(index_dir.absolute()), wmodel="BM25")

## Import Axioms
Here we're listing which axioms we want to use in our experiments.
Because some axioms require API calls or are computationally expensive, we cache all axioms using `ir_axiom`'s tilde operator (`~`).

In [ ]:
from ir_axioms.axiom import (
    ArgUC, QTArg, QTPArg, aSL, PROX1, PROX2, PROX3, PROX4, PROX5, TFC1, TFC3, 
    AND, LEN_AND, M_AND, LEN_M_AND, DIV, LEN_DIV, M_TDC, LEN_M_TDC, STMC1, STMC2, LNC1, TF_LNC, LB1,
    REG, ANTI_REG, ASPECT_REG, ORIG
)
from ir_axioms.integrations.pyterrier.utils import inject_pyterrier

inject_pyterrier(
    index_location=index_dir,
    text_field=None,
    dataset=dataset_name,
)

axioms = [
    ArgUC().cached(cache_dir / "ArgUC"),
    QTArg().cached(cache_dir / "QTArg"),
    QTPArg().cached(cache_dir / "QTPArg"),
    aSL().cached(cache_dir / "aSL"),
    LNC1().cached(cache_dir / "LNC1"),
    TF_LNC().cached(cache_dir / "TF_LNC"),
    LB1().cached(cache_dir / "LB1"),
    PROX1().cached(cache_dir / "PROX1"),
    PROX2().cached(cache_dir / "PROX2"),
    PROX3().cached(cache_dir / "PROX3"),
    PROX4().cached(cache_dir / "PROX4"),
    PROX5().cached(cache_dir / "PROX5"),
    REG().cached(cache_dir / "REG"),
    ANTI_REG().cached(cache_dir / "ANTI_REG"),
    ASPECT_REG().cached(cache_dir / "ASPECT_REG"),
    AND().cached(cache_dir / "AND"),
    LEN_AND().cached(cache_dir / "LEN_AND"),
    M_AND().cached(cache_dir / "M_AND"),
    LEN_M_AND().cached(cache_dir / "LEN_M_AND"),
    DIV().cached(cache_dir / "DIV"),
    LEN_DIV().cached(cache_dir / "LEN_DIV"),
    TFC1().cached(cache_dir / "TFC1"),
    TFC3().cached(cache_dir / "TFC3"),
    M_TDC().cached(cache_dir / "M_TDC"),
    LEN_M_TDC().cached(cache_dir / "LEN_M_TDC"),
    STMC1().cached(cache_dir / "STMC1"),
    STMC2().cached(cache_dir / "STMC2"),
    ORIG(),
]

## KwikSort Re-ranking with Estimating the ORACLE Axiom
We have now defined the axioms with which we want to estimate the ORACLE axiom.
To remind, the ORACLE axiom replicates the perfect ordering induced by human relevance judgments (i.e. from qrels).
We combine the preferences from all axioms in a random forest classifier.
The resulting output preferences can be used with KwikSort to re-rank the top-20 baseline results.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from ir_axioms.tools import MiddlePivotSelection
from ir_axioms.integrations.pyterrier.estimator import EstimatorKwikSortReranker

random_forest = RandomForestClassifier(
    max_depth=3,
)
kwiksort_random_forest = bm25 % 20 >> EstimatorKwikSortReranker(
    axioms=axioms,
    estimator=random_forest,
    index=index_dir,
    dataset=dataset_name,
    text_field=None,
    pivot_selection=MiddlePivotSelection(),
    verbose=True,
)

After setting up the trainable PyTerrier module, we pass in training topics and relevance judgments for training.

In [ ]:
kwiksort_random_forest.fit(dataset_train.get_topics(), dataset_train.get_qrels())

## Experimental Evaluation
Because our axiomatic re-rankers are PyTerrier modules, we can now use PyTerrier's `Experiment` interface to evaluate various metrics and to compare our new approach to the BM25 baseline ranking.
Refer to the PyTerrier [documentation](https://pyterrier.readthedocs.io/en/latest/experiments.html) to learn more about running experiments.
(We concatenate results from the Baseline ranking for the ranking positions after the top-20 using the `^` operator.)

In [ ]:
from pyterrier.pipelines import Experiment
from ir_measures import nDCG, MAP, RR

experiment = Experiment(
    [bm25, kwiksort_random_forest ^ bm25],
    dataset_test.get_topics(),
    dataset_test.get_qrels(),
    [nDCG @ 10, RR, MAP],
    ["BM25", "KwikSort Random Forest"],
    verbose=True,
)
experiment.sort_values(by="nDCG@10", ascending=False, inplace=True)

In [ ]:
experiment

## Extra: Feature Importances
Inspecting the feature importances from the random forest classifier can help to identify axioms that are not used for re-ranking.
If an axiom's feature importance is zero for most of your applications, you may consider omitting it from the ranking pipeline.

In [ ]:
random_forest.feature_importances_